In [7]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + 'FE/FE_total2.csv')


In [8]:
dat.drop(['day','year','KnowledgeTag', 'last_answerCode7','last_answerCode8','last_answerCode9','last_answerCode10'], axis = 1, inplace = True)

In [9]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [10]:
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

In [11]:
# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _test[col] = _test[col].astype('str')

In [12]:
#!pip install catboost
# 가상환경에 패키지 설치 필요할듯
from catboost import CatBoostClassifier

# 하이퍼파라미터 수정할 여지가 많으니 추가나 수정하고 싶은 사람 수정해도 괜찮을듯.
# 일단 지난번에 사용했던 파라미터들 주석으로 들고 옴. 참고하면서 공부하면 좋을듯.

params_cat = {
    "task_type" : "GPU",
    "devices" : '0',
    "random_state": 1998,
    'learning_rate': 0.01, 
    #'bagging_temperature': 0.12172958098369972, 
    'n_estimators': 4000, 
    'max_depth': 11, 
    'random_strength': 18, 
    'l2_leaf_reg': 8.771418009571192e-06, 
    'min_child_samples': 40, 
    'max_bin': 337, 
    'od_type': 'Iter',
    "cat_features" : cat_columns,
}

cat_model = CatBoostClassifier(**params_cat)

# 실제 적용한 모델은 learning_rate, iterations(=n_estimators, 아마도?)만 사용함.
#cat_model = CatBoostClassifier(learning_rate=0.01, random_state = 1998, iterations=4000, task_type="GPU", eval_metric = 'AUC',max_depth = 8)
cat_model.fit(_train, _train_value)


0:	learn: 0.6884465	total: 493ms	remaining: 32m 50s
1:	learn: 0.6840296	total: 1.07s	remaining: 35m 45s
2:	learn: 0.6800003	total: 1.62s	remaining: 36m 4s
3:	learn: 0.6757800	total: 2.19s	remaining: 36m 22s
4:	learn: 0.6714479	total: 2.68s	remaining: 35m 41s
5:	learn: 0.6674477	total: 3.24s	remaining: 35m 58s
6:	learn: 0.6642317	total: 3.48s	remaining: 33m 3s
7:	learn: 0.6602374	total: 4.02s	remaining: 33m 24s
8:	learn: 0.6564595	total: 4.54s	remaining: 33m 31s
9:	learn: 0.6531480	total: 4.81s	remaining: 31m 58s
10:	learn: 0.6495578	total: 5.38s	remaining: 32m 29s
11:	learn: 0.6460122	total: 5.87s	remaining: 32m 30s
12:	learn: 0.6419874	total: 6.34s	remaining: 32m 23s
13:	learn: 0.6385038	total: 6.8s	remaining: 32m 15s
14:	learn: 0.6352366	total: 7.3s	remaining: 32m 20s
15:	learn: 0.6318170	total: 7.82s	remaining: 32m 27s
16:	learn: 0.6285451	total: 8.34s	remaining: 32m 33s
17:	learn: 0.6253752	total: 8.77s	remaining: 32m 20s
18:	learn: 0.6222200	total: 9.31s	remaining: 32m 31s
19:	lea

In [13]:
# valid score를 확인해봄.
# train 대비 valid score가 떨어짐. 과적합이 일어남. 마찬가지로 규제하는 하이퍼파라미터 필요.
from sklearn.metrics import accuracy_score, roc_auc_score

print('train score')
_train_pred = cat_model.predict_proba(_train)[:,1]
print(roc_auc_score(_train_value, _train_pred)) # auc
print(accuracy_score(_train_value, np.where(_train_pred >= 0.5, 1, 0))) # acc, 정확도

train score
0.8992931038619022
0.8346582442449512


In [14]:
# submission 제출하기 위한 코드

_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('../output/cat1207.csv', index = False)